In [11]:
# Henry Judkins
# Using the tensorflow dataset/model to identify different kinds of flowers



import my_warnings
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt

# Downloading the flower photos from tensorflow 
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)

# Finding the directory with the flower photos
data_dir = pathlib.Path(archive)

# Updating data_dir to point to the correct sub-directory
data_dir = data_dir / 'flower_photos'

# Defining parameters for the loader
batch_size = 32
img_height = 180
img_width = 180

# Creating the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Creating the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Standardizing data
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Configuring the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Defining parameters for training
num_classes = 5

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),   # condensing data for optimization purposes
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

# Compiling the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Training the model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Epoch 1/3


2025-09-03 16:05:58.975046: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 129777664 exceeds 10% of free system memory.
2025-09-03 16:05:59.529432: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 32444416 exceeds 10% of free system memory.
2025-09-03 16:05:59.552293: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 31002624 exceeds 10% of free system memory.
2025-09-03 16:05:59.728979: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 29047680 exceeds 10% of free system memory.
2025-09-03 16:05:59.789374: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 31002624 exceeds 10% of free system memory.


92/92 ━━━━━━━━━━━━━━━━━━━━ 81s 854ms/step - accuracy: 0.4261 - loss: 1.3184 - val_accuracy: 0.5872 - val_loss: 1.1103
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 61s 662ms/step - accuracy: 0.6080 - loss: 1.0177 - val_accuracy: 0.5926 - val_loss: 1.0121
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 59s 645ms/step - accuracy: 0.6669 - loss: 0.8561 - val_accuracy: 0.6471 - val_loss: 0.9432
